# Teste dos Modelos sem Ajuste

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchinfo
import numpy as np
import sys
import os

from pathlib import Path

sys.path.insert(0, os.path.abspath('src'))

import src.vcpi_util as vcpi_util
from src.constants import device, BATCH_SIZE, WORKERS, PREFETCH, EPOCHS
from src.models import ModernCNN, ImprovedCNN, EfficientCNN, AttentionCNN, train_model, evaluate_model

In [ ]:
PATH_TRAINING_SET = Path("data/train_images").resolve()
PATH_TEST_SET = Path("data/test_images").resolve()

As transformações não são utilizadas neste contexto para testar a eficácia isolada do modelo.

Apenas o redimensionamento é aplicado uma vez que o modelo exige uma entrada de tamanho fixo.

In [ ]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize((32,32)), torchvision.transforms.ToTensor()])

In [ ]:
train_set = torchvision.datasets.ImageFolder(root=PATH_TRAINING_SET, transform = transform)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=WORKERS,
    pin_memory=True,
    prefetch_factor=PREFETCH,
    persistent_workers=True)


test_set = torchvision.datasets.ImageFolder(root=PATH_TEST_SET, transform = transform)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    num_workers=WORKERS,
    pin_memory=True,
    prefetch_factor=PREFETCH,
    persistent_workers=True)

In [ ]:
def inverse_normalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor


def show_bad_preds(model, dataset, classes):
    k = 0
    iters = 0

    preds = []
    ground_truth = []
    imgs = torch.Tensor(52, 3, 128,128)

    iterator = iter(dataset)

    max_iters = test_set.__len__() / BATCH_SIZE
    while k < 52 and iters < max_iters:

        images, targets = next(iterator)
        #print(images[0].shape)
        logits = model(images.to(device))
        
        #print(predictions[0])
        predictions = torch.nn.functional.softmax(logits, dim=1).cpu().detach().numpy()
        for i in range(len(predictions)):

            if np.argmax(predictions[i]) != targets[i] and k < 52:

                preds.append(predictions[i])
                ground_truth.append(targets[i])
                imgs[k, :, :, :] = inverse_normalize(images[i],[3, 128, 128], [0.229, 0.224, 0.225])
                k += 1

        iters += 1

    vcpi_util.plot_predictions(imgs, preds, ground_truth, classes, 13, 4)     

In [ ]:
def model_info(model, train_loader, train_set, test_loader, EPOCHS, name="AttentionCNN"):
    model.to(device)

    torchinfo.summary(model, input_size=(BATCH_SIZE, 3, 32, 32))

    optimizer = torch.optim.Adam(model.parameters())
    loss_fn = torch.nn.CrossEntropyLoss()

    history = train_model(model, train_loader, test_loader, EPOCHS, loss_fn, optimizer)

    best_train = np.asarray(history['train_acc']).argmax()
    best_test = np.asarray(history['val_acc']).argmax()

    print('Best epoch for train accuracy: :', best_train,' Best epoch for test accuracy: :',  best_test)
    print('Test accuracy at epoch ',best_train, ' :', history['train_acc'][best_train], 'Test accuracy at epoch ',best_test, ' :', history['train_acc'][best_test])
    preds = []
    ground_truth = []

    for images, targets in test_loader:

        logits = model(images.to(device))
        preds_sparse = [np.argmax(x) for x in logits.cpu().detach().numpy()]
        preds.extend(preds_sparse)
        ground_truth.extend(targets.numpy())

    vcpi_util.show_confusion_matrix(ground_truth, preds, len(train_set.class_to_idx))

    evaluate_model(model, test_loader)

    vcpi_util.show_history_plus(history, ['train_acc', 'val_acc'])

# Modelo Attention



In [ ]:
model = AttentionCNN(len(train_set.classes))

model_info(model, train_loader, train_set, test_loader, EPOCHS, name="AttentionCNN")

# Modelo Modern

In [ ]:
model = ModernCNN(len(train_set.classes))
model.to(device)

model_info(model, train_loader, train_set, test_loader, EPOCHS, name="ModernCNN")

# Modelo Improved

In [ ]:
model = ImprovedCNN(len(train_set.classes))
model.to(device)

model_info(model, train_loader, train_set, test_loader, EPOCHS, name="ImprovedCNN")

# Modelo Efficient

In [ ]:
model = EfficientCNN(len(train_set.classes))
model.to(device)

model_info(model, train_loader, train_set, test_loader, EPOCHS, name="EfficientCNN")